# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import os
from IPython.display import Markdown

import warnings
warnings.filterwarnings('ignore')

In [2]:
# # account for deprecation of LLM model
# import datetime
# # Get the current date
# current_date = datetime.datetime.now().date()

# # Define the date after which the model should be set to "gpt-3.5-turbo"
# target_date = datetime.date(2024, 6, 12)

# # Set the model variable based on the current date
# if current_date > target_date:
#     llm_model = "gpt-3.5-turbo"
# else:
#     llm_model = "gpt-3.5-turbo-0301"

In [3]:
#!pip install pandas

In [4]:
llm_model = 'gpt-4o'

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to la...


## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [8]:
llm = ChatOpenAI(model_name = 'gpt-4o',
                 temperature=0.9)

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
    )

In [10]:
# create a sequence with pipe operator
chain = prompt | llm | StrOutputParser()

In [11]:
# Run the chain
Markdown(chain.invoke({"product" : "Queen Size pillow"}))

Choosing the best name for a company that makes Queen Size pillows depends on several factors such as your brand's identity, target audience, and overall vision. Here are a few ideas to get you started:

1. **RoyalRest Pillows**
2. **Queen's Comfort**
3. **MajesticSleep Pillows**
4. **RegalDream Pillows**
5. **CrownLuxury Pillows**
6. **QueenQuilt Pillows**
7. **NobleNights Pillows**
8. **RoyalSlumber Pillows**
9. **Queen's Haven Pillows**
10. **SupremeSlumber Pillows**

Remember to check the availability of the name for domain registration and ensure it is not already in use by another business.

## SimpleSequentialChain

In [12]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

In [13]:
llm = ChatOpenAI(model_name = 'gpt-4o',
                 temperature = 0.9)

In [14]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
    )

second_prompt = ChatPromptTemplate.from_template(
    "Convert only company name into german : {company_name}, choose any one and use your creativity to describe"
    )

In [15]:
overall_simple_chain = (
    first_prompt | llm | StrOutputParser() | second_prompt | llm | StrOutputParser()
)

In [16]:
console_handler = ConsoleCallbackHandler()

In [17]:
Markdown(overall_simple_chain.invoke({"product" : "pillows"}))

Let's go with **CloudRest** and convert it to German as **WolkenRuhe**.

**WolkenRuhe** – A Symphony of Comfort in Every Slumber

When you lay your head down at night, you deserve nothing but the very best in comfort and serenity. At **WolkenRuhe**, we transform every sleeping moment into a rejuvenating escape. Our name, directly translating to "Cloud Rest," embodies the essence of weightlessness and supreme relaxation. Imagine sinking into a pillow so soft and supportive, it feels like you're floating on a cloud. 

**WolkenRuhe** isn’t just about pillows; it's about creating the ultimate sleep experience. We handcraft each pillow using only the finest materials, ensuring that each product stands up to our rigorous standards of quality. Every stitch, every fiber, and every cushion is designed with your deepest comfort in mind.

Our belief is simple: a well-rested you is a happier, healthier you. That’s why we’ve dedicated ourselves to innovating the perfect blend of softness and support. Whether you’re a back, side, or stomach sleeper, **WolkenRuhe** has the ideal pillow to cradle you through the night.

Join us in our mission to redefine sleep. Experience the luxury of floating on a cloud, night after night, with **WolkenRuhe**. Because when it comes to your rest, only the best will do.

## SequentialChain

In [18]:
ac_review = """
If you are looking for a complete package, then look nowhere other than Daikin.

I purchased the AC 1.5 mnths ago, and until now I am impressed with what it has to offer. Installation went super smooth. The installation guys came within the same day. The installation process started, and it was done in 3 hours max.
Also, for those people who are saying that they are charging money even when installation is free, you guys have no idea about how AC installation is done.The extra charges can vary heavily depending on the amount of material needed including pipe, tape and so on. In my case, it cost me 5000INR, and I am super happy with it.The build quality of the AC is excellent. The internal unit looks sleek and modern. The AC functions as advertised. It cools very nicely.

Overall Value
The overall value of the AC is just excellent. It is affordable and enables you to enjoy with minimal electricity bills.
"""

In [19]:
llm = ChatOpenAI(model=llm_model,
                 temperature=0.9)

In [21]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

# prompt template 2 : summarize
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Summary}"
)

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {Translated_Review}"
    "\n\nLanguage: {mylanguage}"
)

In [23]:
from langchain_core.runnables import RunnableMap

In [24]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message

runnable1 = RunnableMap({
    "English_Review" : first_prompt | llm | StrOutputParser(),
    "mylanguage": lambda x: x["mylanguage"]
    })
runnable2 = RunnableMap({
    "Summary" : second_prompt | llm | StrOutputParser(),
    "mylanguage": lambda x: x["mylanguage"]
    })
runnable3 = RunnableMap({
    "Translated_Review" : third_prompt | llm | StrOutputParser(),
    "mylanguage": lambda x: x["mylanguage"]
                         })
runnable4 = RunnableMap({
    "Final_review" : fourth_prompt | llm | StrOutputParser(),
    })

In [25]:
seq = runnable1 | runnable2 | runnable3 | runnable4

In [26]:
input_data = {"Review" : ac_review, "mylanguage" : "hindi"}

In [27]:
result = seq.invoke(input_data, config={'callbacks': [console_handler]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "Review": "\nIf you are looking for a complete package, then look nowhere other than Daikin.\n\nI purchased the AC 1.5 mnths ago, and until now I am impressed with what it has to offer. Installation went super smooth. The installation guys came within the same day. The installation process started, and it was done in 3 hours max.\nAlso, for those people who are saying that they are charging money even when installation is free, you guys have no idea about how AC installation is done.The extra charges can vary heavily depending on the amount of material needed including pipe, tape and so on. In my case, it cost me 5000INR, and I am super happy with it.The build quality of the AC is excellent. The internal unit looks sleek and modern. The AC functions as advertised. It cools very nicely.\n\nOverall Value\nThe overall value of the AC is just excellent. It is affordable and enables you to enjoy with minimal electrici

In [28]:
Markdown(result['Final_review'])

समीक्षा अंग्रेजी में है लेकिन आपकी प्रतिक्रिया में हिंदी का उपयोग करने का अनुरोध किया गया है। कृपया इस समीक्षा के किसी विशेष बिंदु पर ध्यान केंद्रित करें या कोई खास प्रश्न उठाएँ। हमें आपसे और जानने में खुशी होगी। धन्यवाद!

## Router Chain

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [30]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [31]:
chain = (
    PromptTemplate.from_template(
        """
        Given the user question below, classify it as either being about `physics`, `maths`, `history`, `Computer science`.
        
        Do not respond with more than one word.

        <question>
        {input}
        </question>

        Classification:"""
    )
    | llm
    | StrOutputParser()
)

In [32]:
chain.invoke({'input' : "What is Algebra"})

'maths'

#### Creating sub chains

In [33]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

general_template = """Respond to the following question:
                   Here is a question:
                   {input}
                   """

In [34]:
# defining prompt templates
physics_chain = PromptTemplate.from_template(physics_template) | llm | StrOutputParser()
maths_chain = PromptTemplate.from_template(math_template) | llm | StrOutputParser()
history_chain = PromptTemplate.from_template(history_template) | llm | StrOutputParser()
cs_chain = PromptTemplate.from_template(computerscience_template) | llm | StrOutputParser()
general_chain = PromptTemplate.from_template(general_template) | llm | StrOutputParser()

### Using a custom function (Recommended)

In [35]:
def route(info):
    print(info)
    if "physics" in info["topic"].lower():
        return physics_chain
    elif "maths" in info["topic"].lower():
        return maths_chain
    elif "history" in info["topic"].lower():
        return history_chain
    elif "computer science" in info["topic"].lower():
        return cs_chain
    else:
        return general_chain

In [36]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "input": lambda x: x["input"]} | RunnableLambda(
    route
)

In [37]:
Markdown(full_chain.invoke({"input": "What is difference between speed and acceleration"}))

{'topic': 'physics', 'input': 'What is difference between speed and acceleration'}


Sure, I'd be happy to explain the difference between speed and acceleration!

**Speed** is a measure of how fast an object is moving. It tells you the distance an object travels over a certain period of time. Speed is a scalar quantity, which means it only has magnitude and no direction. For example, if a car is traveling at 60 miles per hour, that number tells you how fast the car is going, but not the direction in which it is moving.

**Acceleration**, on the other hand, is a measure of how quickly an object's speed is changing. It tells you the rate at which the velocity of an object is changing over time. Acceleration is a vector quantity, which means it has both magnitude and direction. For example, if a car is increasing its speed by 10 miles per hour every second, it has an acceleration of 10 miles per hour per second in the direction of its motion.

In summary:
- **Speed**: How fast an object is moving (scalar).
- **Acceleration**: How quickly the speed of an object is changing (vector).

I hope that helps! If you have any more questions, feel free to ask.

### Using a RunnableBranch

In [38]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "physics" in x["topic"].lower(), physics_chain),
    (lambda x: "maths" in x["topic"].lower(), maths_chain),
    (lambda x: "history" in x["topic"].lower(), history_chain),
    (lambda x: "computer science" in x["topic"].lower(), cs_chain),
    general_chain,
)
full_chain = {"topic": chain, "input": lambda x: x["input"]} | branch

In [39]:
Markdown(full_chain.invoke({"input": "Who is lord sri krishna?"}))

Lord Sri Krishna is a central figure in Hinduism, revered as the eighth avatar (incarnation) of the god Vishnu. He is a multifaceted deity, embodying various roles and attributes, and his life and teachings have had a profound impact on Indian culture, philosophy, and religion.

### Historical and Mythological Context
1. **Birth and Early Life**: According to Hindu tradition, Krishna was born in the Yadava clan to Devaki and Vasudeva in the city of Mathura. His birth is celebrated as Janmashtami. To protect him from the tyrannical king Kamsa, who was prophesied to be killed by Devaki's eighth son, Krishna was secretly transported to Gokul, where he was raised by foster parents Nanda and Yashoda.

2. **Childhood and Youth**: Krishna's childhood is filled with miraculous events and playful exploits, often depicted in texts like the Bhagavata Purana. He is known for his mischievous behavior, such as stealing butter and playing pranks on the gopis (milkmaids). These stories highlight his divine nature and his role as a protector of the innocent.

3. **Role in the Mahabharata**: Krishna plays a crucial role in the Indian epic, the Mahabharata. He serves as a charioteer and advisor to the Pandava prince Arjuna. The Bhagavad Gita, a 700-verse Hindu scripture that is part of the Mahabharata, is a conversation between Krishna and Arjuna on the battlefield of Kurukshetra. In this dialogue, Krishna imparts spiritual wisdom and guidance, addressing complex issues of duty, righteousness, and the nature of the self.

4. **Philosophical Teachings**: Krishna's teachings in the Bhagavad Gita cover various paths to spiritual realization, including Karma Yoga (the path of selfless action), Bhakti Yoga (the path of devotion), and Jnana Yoga (the path of knowledge). His discourse emphasizes the importance of performing one's duty without attachment to the results and surrendering to the divine will.

5. **Later Life and Legacy**: After the Mahabharata war, Krishna returned to his kingdom in Dwaraka. His later life is marked by his role as a king and a diplomat. According to tradition, Krishna eventually left his mortal body, an event known as "Krishna's departure," which marks the beginning of the Kali Yuga, the current age in Hindu cosmology.

### Cultural and Religious Significance
- **Devotional Worship**: Krishna is worshipped in various forms across India and the world. He is often depicted as a divine lover, a playful child, and a wise teacher. Temples dedicated to Krishna, such as the famous Jagannath Temple in Puri and the ISKCON (International Society for Krishna Consciousness) movement, attract millions of devotees.

- **Festivals**: Major festivals celebrating Krishna include Janmashtami (his birth), Holi (the festival of colors, associated with his playful nature), and Rasa Lila (celebrating his divine love with the gopis).

- **Literature and Arts**: Krishna's life and exploits have inspired a vast body of literature, music, dance, and art. His stories are recounted in texts like the Bhagavata Purana, the Harivamsa, and various regional literatures. Classical Indian dance forms like Bharatanatyam and Kathak often depict episodes from Krishna's life.

In summary, Lord Sri Krishna is a complex and multifaceted deity whose life and teachings continue to influence Hindu thought, culture, and spirituality. His role as a divine guide, protector, and lover makes him one of the most beloved and revered figures in Hinduism.